# REBOOTING A SERVER

<div class="alert alert-block alert-info">
<i>Hello User.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> This is the example to <b>perform rebooting of a server.</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "https://github.com/HewlettPackard/python-ilorest-library" style ="">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>


<div class = "alert alert-block alert-success">
<h1>1</h1>    
In the beginning, we are having some import statements that are actually providing us with the required modules and libraries.
for proper execution of our code.</div>

In [1]:

import sys
import json
import getpass
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError


<div class = "alert alert-block alert-success">
    <h1>2</h1>   
In below cell is the reboot_server function.
Current cell is the overview of it, utimately at the end this will reboot the server and provide the response for the same.
    <br>
    <i>Let me give you a walk through of it!</i>
    <br>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to obtain the power metrics information.

1. In the <b>reboot server</b> function we are first using <b>system_members_response</b> variable to obtain information about the path at which server related information is stored by using couple of get requests.
<br>
2. If the resource directory was accessible then we simply search for <b><i>'#ComputerSystem'</i></b> key (as this is the place comprising of server related information inside the <b><i>instance['odata.type']</i></b> dictionary and finally making a get request to that location.
<br>
    
3. If we obtain systems_response successfully then a <b>ressettype</b> list is created specifying the parameters that how we want to restart.
    

4. Later both the parameters are passed into a new dictionary named body and post request is created to make changes on the server side.
    A successful <a href = "https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods/POST">HTTP post</a> request will set the values at server end so that a server gets rebooted.
    

5. Lastly, we also save response status inside <b>resp</b> variable and check what is the status code returned to confirm whether the post request was success or not. If success then response by server is printed and if not then error message is printed.

 </div>

In [ ]:
def reboot_server(_redfishobj):

    systems_members_response = None

    resource_instances = get_resource_directory(_redfishobj.get_resource_directory)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        systems_uri = _redfishobj.root.obj['Systems']['@odata.id']
        systems_response = _redfishobj.get(systems_uri)
        systems_uri = next(iter(systems_response.obj['Members']))['@odata.id']
        systems_response = _redfishobj.get(systems_uri)
    else:
        for instance in resource_instances:
            #Use Resource directory to find the relevant URI
            if '#ComputerSystem.' in instance['@odata.type']:
                systems_uri = instance['@odata.id']
                systems_response = _redfishobj.get(systems_uri)

    if systems_response:
        system_reboot_uri = systems_response.obj['Actions']['#ComputerSystem.Reset']['target']
        body = dict()
        resettype = ['ForceRestart','GracefulRestart']
        body['Action'] = 'ComputerSystem.Reset'
        for reset in resettype:
            if reset.lower() == "forcerestart":
                body['ResetType'] = "ForceRestart"
                resp = _redfishobj.post(system_reboot_uri, body)
            elif reset.lower() == "gracefulrestart":
                body['ResetType'] = "GracefulRestart"
                resp = _redfishobj.post(system_reboot_uri, body)

        #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
        #error message to see what went wrong
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                    sort_keys=True))
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO Extended "
                                 "Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success!\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))

<div class="alert alert-block alert-success">
<h1>3</h1>   
<b>The Main function: </b> PLEASE ENTER YOU LOGIN CREDENTIALS HERE TO PROCEED FURTHER<br>
<ul>
        
<li>Once your credentials that are enter everything a redfish object is created with the help of RedfishClient class and a session is created and now you will be logged in to the server.</li>
<li>If server is down or Unreachable we get server not reachable message!</li>
<li>If not then we make a call to <b>reboot_server function</b> to perform rebooting of server.</li>
<li>After getting required information <i><b>Redfish.logout()</b></i>closses the particular session and logs us out of the system.</li>
    </ul>
</div>

In [ ]:
if __name__ == "__main__":
    
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Name: ")
    LOGIN_PASSWORD = getpass.getpass("Enter password: ")

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    reboot_server(REDFISHOBJ)
    REDFISHOBJ.logout()

<div class = "alert alert-block alert-warning">
    <b>When running on the server locally use the following commented values</b><br>
 SYSTEM_URL = None<br>
 LOGIN_ACCOUNT = None<br>
 LOGIN_PASSWORD = None<br>

   <b>When running remotely</b> connect using the secured (https://) address,
     account name, and password to send https requests
    <br>
     SYSTEM_URL acceptable examples:<br>
     https://10.0.0.100<br>
     https://ilo.hostname
</div>

<div class = "alert alert-block alert-success">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>